## Дерево решений

Задание
1. Там, где написано "Ваш код", нужно реализовать метод или часть метода
2. Там, где написано "Что делает этот блок кода?", нужно разобраться в блоке кода и в комментарии написать, что он делает
3. Добиться, чтобы в пункте "Проверка скорости работы" Ваша реализация работала чуть быстрее, чем у дерева из sklearn (это возможно, так как мы реализуем только малую часть функциональности)
4. Добиться, чтобы в пункте "Проверка качества работы" Ваша реализация работала так же или качественнее, чем у дерева из sklearn
5. Применить реализованное дерево решений для задачи Titanic на kaggle. Применить для той же задачи дерево решений из sklearn. Применить кросс-валидацию для подбора параметров. Сравнить с результатами предыдущих моделей. Если результат улучшился - сделать сабмит. Написать отчет о результатах.

In [41]:
from time import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy import optimize
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

%matplotlib inline

In [2]:
class MyDecisionTree:
    def __init__(self, depth=0, max_depth=6, criterion = 'gini', min_impurity_split = 10**-7):
        self.depth = depth
        self.max_depth = max_depth
        self.criterion = criterion
        self.min_impurity_split = min_impurity_split
        
    def impurity_score(self, y):
        if y is None or y.shape[0] == 0:
            return 0
        # вычисляем вероятности для всех уникальных классов
        unique, counts = np.unique(y, return_counts=True)
        probs = counts/len(y)
        impurity = 0
        if self.criterion == 'gini':
            for val in probs:
                impurity += val*(1-val)
        elif self.criterion == 'entropy':
            for val in probs:
                impurity += -val*np.log2(val)
        return impurity
    
    def information_gain(self, x, y, split):
        y_left = y[x < split]
        y_right = y[x >= split]
        N = len(y)
        return self.impurity_score(y) - ((len(y_left)/N) * self.impurity_score(y_left) + \
                                         (len(y_right)/N) * self.impurity_score(y_right)) 

    def find_split(self, data, col):
        data = data.sort_values(col)
        x_values = data[col].values
        y_values = data[self.target].values
        
        #Находим индексты где значение значение различное от соседней. По ним будем проверять split
        boundaries = np.nonzero(y_values[:-1] != y_values[1:])[0]
        best_split = None
        max_ig = 0
        for b in boundaries:
            split = (x_values[b] + x_values[b+1])/2
            ig = self.information_gain(x_values, y_values, split)
            if ig > max_ig:
                max_ig = ig
                best_split = split
        return max_ig, best_split
    


    def fit(self, data, target):
        self.target = target
        if data.shape[0] == 1:
            self.col = None
            self.split = None
            self.left = None
            self.right = None
            self.prediction = data[target].max()
        else:
            cols = [col for col in data.columns if col != self.target]
            max_ig = 0
            best_col = None
            best_split = None
            for col in cols:
                ig, split = self.find_split(data, col)
                if ig > max_ig:
                    max_ig = ig
                    best_col = col
                    best_split = split

            if max_ig <= self.min_impurity_split:
                self.col = None
                self.split = None
                self.left = None
                self.right = None
                self.prediction = np.round(data[self.target].mean())
            else:
                self.col = best_col
                self.split = best_split

                if self.depth == self.max_depth:
                    self.left = None
                    self.right = None
                    self.prediction = np.round(data[self.target].mean())

                else:
                    DataLeft = data.loc[data[best_col] < best_split]
                    self.left = MyDecisionTree(self.depth + 1, self.max_depth)
                    self.left.fit(DataLeft, self.target)

                    DataRight = data.loc[data[best_col] >= best_split]
                    self.right = MyDecisionTree(self.depth + 1, self.max_depth)
                    self.right.fit(DataRight, self.target)

    def predict_one(self, x):
        if self.col is not None and self.split is not None:
            feature = x[self.col]
            if feature < self.split:
                if self.left:
                    p = self.left.predict_one(x)
                else:
                    p = self.prediction
            else:
                if self.right:
                    p = self.right.predict_one(x)
                else:
                    p = self.prediction
        else:
            p = self.prediction
        return p

    def predict(self, X):
        N = len(X)
        P = np.zeros(N)
        for idx, row in X.iterrows():
            P[idx] = self.predict_one(row)
        return P


**Загрузим предобработанные данные Титаник**

In [13]:
train_data = pd.read_csv('input/train_preprocessed.csv')
test_data = pd.read_csv('input/test_preprocessed.csv')

y_column = 'Survived'
x_columns = [col for col in train_data.columns if col != 'Survived']

## Проверка скорости работы

In [14]:
my_clf = MyDecisionTree(max_depth = 3, criterion = 'gini')
clf = DecisionTreeClassifier(max_depth=3, criterion = 'gini')

In [16]:
t1 = time()
my_clf.fit(train_data, target = 'Survived')
t2 = time()
print(t2 - t1)

t1 = time()
clf.fit(train_data[x_columns], train_data[y_column])
t2 = time()
print(t2 - t1)

5.814981937408447
0.014863252639770508
